# Data load & prepare libs

In [1]:
!pip install catboost -q
!pip install transformers -q
!pip install datasets -q

In [2]:
# Загружаем библиотеки
import numpy as np
import pandas as pd
import itertools
import random
import re
from tqdm import notebook
import time
import itertools


# Модели и инструменты
from scipy.special import softmax
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LassoLars, RidgeClassifier
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from catboost import CatBoostClassifier, CatBoostRegressor

# Нейронки
import torch
import transformers
from transformers import AutoTokenizer, AutoModel
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import TrainingArguments, Trainer

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
# Проверяем доступность GPU
torch.cuda.is_available()

True

In [4]:
device = torch.device("cuda")

In [5]:
df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")

In [6]:
df_train.head(5)

,Unnamed: 0,text,rate
0,0,Story of a man who has unnatural feelings for ...,3
1,1,Robert DeNiro plays the most unbelievably inte...,1
2,2,"I saw the capsule comment said ""great acting.""...",1
3,3,If I had not read Pat Barker's 'Union Street' ...,4
4,4,This fanciful horror flick has Vincent Price p...,4


In [7]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False

SEED = 42
seed_all(SEED)

In [8]:
df_val, df_test = train_test_split(df_test, test_size=.5, random_state=SEED)

train_text = df_train['text'].values
y_train = df_train['rate'].values

val_text = df_val['text'].values
y_val = df_val['rate'].values

test_text = df_test['text'].values
y_test = df_test['rate'].values

In [9]:
def reg_cls_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    bin_true = np.where(y_true > 5, 1, 0)
    bin_pred = np.where(y_pred > 5, 1, 0)
    acc = accuracy_score(bin_true, bin_pred)
    return mse, mae, acc

def print_metrics(y_true, y_pred):
    mse, mae, acc = reg_cls_metrics(y_true, y_pred)
    print(f'mse: {mse}\nmae: {mae}\nacc: {acc}')

# Pertained -> regressor/cls

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")
model = model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def namestr(obj):
    namespace = globals()
    return [name for name in namespace if namespace[name] is obj][0]
    
def get_embedings(model, tokenizer, df_corpus, batch_size=100):

    start_time = time.time()
    tokenized = df_corpus['text'].apply(lambda x: tokenizer.encode(x, 
                                                        max_length=512,
                                                        padding='max_length',
                                                        truncation=True,
                                                        add_special_tokens=True))
    tokenized = np.array(list(tokenized))
    attention_mask = np.where(tokenized != 0, 1, 0)
    print(f'Tokenized time for {namestr(df_corpus)}: {time.time() - start_time:.2f}s')

    embeddings = []
    for i in notebook.tqdm(range(tokenized.shape[0] // batch_size)):
        slice_top = batch_size*(i+1)
        if slice_top == tokenized.shape[0] // batch_size * batch_size:
            slice_top = tokenized.shape[0]
        batch = torch.LongTensor(tokenized[batch_size*i:slice_top])
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:slice_top])
        batch = batch.to(device)
        attention_mask_batch = attention_mask_batch.to(device)
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        batch_embeddings = batch_embeddings[0][:,0,:].to('cpu')
        embeddings.append(batch_embeddings.numpy())
    return np.concatenate(embeddings)

In [ ]:
train_features = get_embedings(model, tokenizer, df_train, batch_size=100)
val_features = get_embedings(model, tokenizer, df_val, batch_size=100)

Tokenized time for df_train: 47.72s


  0%|          | 0/250 [00:00<?, ?it/s]

Tokenized time for df_val: 20.67s


  0%|          | 0/125 [00:00<?, ?it/s]

In [ ]:
ls_result = []

linreg = LinearRegression().fit(train_features, y_train)
pred = linreg.predict(val_features)
print('LinearRegression\n' + '-'*20)
print_metrics(y_val, pred)
mse, mae, acc = reg_cls_metrics(y_val, pred)
ls_result.append(['LinearRegression', mse, mae, acc])

LinearRegression
--------------------
mse: 4.8134902063989635
mae: 1.73754529296875
acc: 0.86424


In [ ]:
res_score = np.inf
for alpha in notebook.tqdm(np.logspace(-6, 2, num=20)):
    model = Lasso(alpha=alpha, random_state=SEED).fit(train_features, y_train)
    pred = model.predict(val_features)
    score = mean_squared_error(y_val, pred)
    if res_score > score:
        res_score = score
        alpha_best = alpha
lasso = Lasso(alpha=alpha_best, random_state=SEED).fit(train_features, y_train)
pred = lasso.predict(val_features)
print(f'Lasso\nalpha: {alpha_best}\n{"-"*20}')
print_metrics(y_val, pred)
mse, mae, acc = reg_cls_metrics(y_val, pred)
ls_result.append(['Lasso', mse, mae, acc])

  0%|          | 0/20 [00:00<?, ?it/s]

Lasso
alpha: 0.00012742749857031334
--------------------
mse: 4.791671351353627
mae: 1.7358503594398498
acc: 0.8648


In [ ]:
res_score = np.inf
for alpha in notebook.tqdm(np.logspace(-6, 2, num=100)):
    model = Ridge(alpha=alpha, random_state=SEED).fit(train_features, y_train)
    pred = model.predict(val_features)
    score = mean_squared_error(y_val, pred)
    if res_score > score:
        res_score = score
        alpha_best = alpha
ridge = Ridge(alpha=alpha_best, random_state=SEED).fit(train_features, y_train)
pred = ridge.predict(val_features)
print(f'Ridge\nalpha: {alpha_best}\n{"-"*20}')
print_metrics(y_val, pred)
mse, mae, acc = reg_cls_metrics(y_val, pred)
ls_result.append(['Ridge', mse, mae, acc])

  0%|          | 0/100 [00:00<?, ?it/s]

Ridge
alpha: 6.135907273413176
--------------------
mse: 4.776531610744256
mae: 1.734499118461609
acc: 0.86704


In [ ]:
model = CatBoostRegressor(verbose=False, 
                        random_state=SEED)
model.fit(train_features, y_train, eval_set=(val_features, y_val))
pred = model.predict(val_features)
print(f'CatBoostRegressor\n{"-"*20}')
print_metrics(y_val, pred)
mse, mae, acc = reg_cls_metrics(y_val, pred)
ls_result.append(['CatBoostRegressor', mse, mae, acc])

CatBoostRegressor
--------------------
mse: 4.81536900686281
mae: 1.6962333202259037
acc: 0.85592


In [ ]:
res_score = np.inf
for n_neighbors in notebook.tqdm(range(2, 13)):
    model = KNeighborsRegressor(n_neighbors=n_neighbors).fit(train_features, y_train)
    pred = model.predict(val_features)
    score = mean_squared_error(y_val, pred)
    if res_score > score:
        res_score = score
        n_neighbors_best = n_neighbors
neigh = KNeighborsRegressor(n_neighbors=n_neighbors_best).fit(train_features, y_train)
pred = neigh.predict(val_features)
print(f'KNeighborsRegressor\nn_neighbors_best: {n_neighbors_best}\n{"-"*20}')
print_metrics(y_val, pred)
mse, mae, acc = reg_cls_metrics(y_val, pred)
ls_result.append(['KNeighborsRegressor', mse, mae, acc])

  0%|          | 0/11 [00:00<?, ?it/s]

KNeighborsRegressor
n_neighbors_best: 12
--------------------
mse: 6.872704444444445
mae: 2.0732266666666668
acc: 0.79024


In [ ]:
params_list = list(itertools.product(np.logspace(-6, 1, num=10),
                                     np.linspace(0.1, 0.9, num=9)))
res_score = np.inf
for alpha, l1_ratio in notebook.tqdm(params_list):
    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=SEED).fit(train_features, y_train)
    pred = model.predict(val_features)
    score = mean_squared_error(y_val, pred)
    if res_score > score:
        res_score = score
        alpha_best = alpha
        l1_ratio_best = l1_ratio
elastic = ElasticNet(alpha=alpha_best, l1_ratio=l1_ratio_best, random_state=SEED).fit(train_features, y_train)
pred = elastic.predict(val_features)
print(f'ElasticNet\nalpha: {alpha_best}\nl1_ratio: {l1_ratio_best}\n{"-"*20}')
print_metrics(y_val, pred)
mse, mae, acc = reg_cls_metrics(y_val, pred)
ls_result.append(['ElasticNet', mse, mae, acc])

  0%|          | 0/90 [00:00<?, ?it/s]

ElasticNet
alpha: 0.00021544346900318845\l1_ratio: 0.1
--------------------
mse: 4.776438493778181
mae: 1.7341281745529176
acc: 0.8668


In [ ]:
params_list = list(itertools.product(np.logspace(-6, 1, num=10),
                                     [True, False]))
res_score = np.inf
for alpha, normalize in notebook.tqdm(params_list):
    model = LassoLars(alpha=alpha, normalize=normalize, random_state=SEED).fit(train_features, y_train)
    pred = model.predict(val_features)
    score = mean_squared_error(y_val, pred)
    if res_score > score:
        res_score = score
        alpha_best = alpha
        normalize_best = normalize
elastic = LassoLars(alpha=alpha_best, normalize=normalize_best, random_state=SEED).fit(train_features, y_train)
pred = elastic.predict(val_features)
print(f'LassoLars\nalpha: {alpha_best}\nnormalize: {normalize_best}\n{"-"*20}')
print_metrics(y_val, pred)
mse, mae, acc = reg_cls_metrics(y_val, pred)
ls_result.append(['LassoLars', mse, mae, acc])

  0%|          | 0/20 [00:00<?, ?it/s]

LassoLars
alpha: 1e-06
ormalize: False
--------------------
mse: 4.87488632791581
mae: 1.7598282445812226
acc: 0.86536


In [ ]:
res_score = np.inf
for C in notebook.tqdm(np.logspace(-6, 2, num=20)):
    model = LinearSVR(C=C, dual=False, loss='squared_epsilon_insensitive', random_state=SEED).fit(train_features, y_train)
    pred = model.predict(val_features)
    score = mean_squared_error(y_val, pred)
    if res_score > score:
        res_score = score
        C_best = C
lin_svm = LinearSVR(C=C_best, dual=False, loss='squared_epsilon_insensitive', random_state=SEED).fit(train_features, y_train)
pred = lin_svm.predict(val_features)
print(f'LinearSVR\nC: {C_best}\n{"-"*20}')
print_metrics(y_val, pred)
mse, mae, acc = reg_cls_metrics(y_val, pred)
ls_result.append(['LinearSVR', mse, mae, acc])

  0%|          | 0/20 [00:00<?, ?it/s]

LinearSVR
C: 0.11288378916846883
--------------------
mse: 4.77960358947158
mae: 1.7343398406819766
acc: 0.86648


In [ ]:
svm = SVR().fit(train_features, y_train)
pred = svm.predict(val_features)
print(f'SVR\n{"-"*20}')
print_metrics(y_val, pred)
mse, mae, acc = reg_cls_metrics(y_val, pred)
ls_result.append(['SVR', mse, mae, acc])

SVR
--------------------
mse: 5.015660732741902
mae: 1.768242034085415
acc: 0.86304


In [ ]:
y_val_bin = np.where(y_val > 5, 1, 0)
y_train_bin = np.where(y_train > 5, 1, 0)
res_score = np.inf
for alpha in notebook.tqdm(np.logspace(-6, 2, num=100)):
    model = RidgeClassifier(alpha=alpha, random_state=SEED).fit(train_features, y_train_bin)
    pred = model.predict(val_features)
    score = mean_squared_error(y_val_bin, pred)
    if res_score > score:
        res_score = score
        alpha_best = alpha
ridge = RidgeClassifier(alpha=alpha_best, random_state=SEED).fit(train_features, y_train_bin)
pred_bin = ridge.predict(val_features)
acc = accuracy_score(y_val_bin, pred_bin)
print(f'RidgeClassifier\nalpha: {alpha_best}\n{"-"*20}\nacc: {acc}')


  0%|          | 0/100 [00:00<?, ?it/s]

RidgeClassifier
alpha: 6.135907273413176
--------------------
acc: 0.87392


In [ ]:
pd.DataFrame(ls_result, columns=['model', 'mse', 'mae', 'acc'])

,model,mse,mae,acc
0,LinearRegression,4.813490,1.737545,0.86424
1,Lasso,4.791671,1.735850,0.86480
2,Ridge,4.776532,1.734499,0.86704
3,CatBoostRegressor,4.815369,1.696233,0.85592
4,KNeighborsRegressor,6.872704,2.073227,0.79024
5,ElasticNet,4.776438,1.734128,0.86680
6,LassoLars,4.874886,1.759828,0.86536
7,LinearSVR,4.779604,1.734340,0.86648
8,SVR,5.015661,1.768242,0.86304


Best result shows Ridge regressor, separate model for binary classify shows better results

In [ ]:
res_score = np.inf
for alpha in notebook.tqdm(np.logspace(-6, 3, num=100)):
    model = Pipeline([('scaler', StandardScaler()), ('ridge', Ridge(alpha=alpha, random_state=SEED))])
    model.fit(train_features, y_train)
    pred = model.predict(val_features)
    score = mean_squared_error(y_val, pred)
    if res_score > score:
        res_score = score
        alpha_best = alpha
ridge = Pipeline([('scaler', StandardScaler()), ('ridge', Ridge(alpha=alpha_best, random_state=SEED))])
ridge.fit(train_features, y_train)
pred = ridge.predict(val_features)
print(f'Ridge (scale) \nalpha: {alpha_best}\n{"-"*20}')
print_metrics(y_val, pred)

  0%|          | 0/100 [00:00<?, ?it/s]

Ridge (scale) 
alpha: 533.6699231206302
--------------------
mse: 4.780092177648449
mae: 1.7353514581680298
acc: 0.86704


# Fine tune (classification)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=8).to(device)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [ ]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)
tokens_val = tokenizer.batch_encode_plus(
    val_text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)

In [ ]:
labels2id = {1:0, 2:1, 3:2, 4:3, 7:4, 8:5, 9:6, 10:7}
id2labels = {0:1, 1:2, 2:3, 3:4, 4:7, 5:8, 6:9, 7:10}

y_train_cls = np.array([labels2id[i] for i in y_train])
y_val_cls = np.array([labels2id[i] for i in y_val])

In [ ]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)
    
train_dataset = Data(tokens_train, y_train_cls)
val_dataset = Data(tokens_val, y_val_cls)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids.flatten()
    labels = np.array([id2labels[i] for i in labels])
    raw_preds = pred.predictions
    preds = raw_preds.argmax(-1)
    preds = np.array([id2labels[i] for i in preds])
    mse, mae, acc = reg_cls_metrics(labels, preds)

    preds_w = np.array([sum([id2labels[i] * w for i, w in enumerate(softmax(arr))]) for arr in raw_preds])
    mse_w, mae_w, acc_w = reg_cls_metrics(labels, preds_w)
    return {'mse': mse, 'mae': mae, 'acc': acc, 'mse_w': mse_w, 'mae_w': mae_w, 'acc_w': acc_w}

In [ ]:
training_args = TrainingArguments(
    output_dir = '/content/distilbert-base-uncased-cls', #Выходной каталог
    num_train_epochs = 5, #Кол-во эпох для обучения
    per_device_train_batch_size = 32, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 32, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = '/content/logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=SEED)

In [ ]:
trainer = Trainer(model=model,
                  tokenizer = tokenizer,
                  args = training_args,
                  train_dataset = train_dataset,
                  eval_dataset = val_dataset,
                  compute_metrics = compute_metrics)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 25000
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3910
  Number of trainable parameters = 66959624


Epoch,Training Loss,Validation Loss,Mse,Mae,Acc,Mse W,Mae W,Acc W
1,1.521100,1.361270,4.119600,1.157680,0.920880,2.890535,1.168956,0.919360
2,1.302400,1.315891,3.328880,1.044240,0.927920,2.615739,1.098636,0.925600
3,1.208700,1.288532,3.332400,1.017680,0.931040,2.521636,1.042900,0.927840
4,1.139400,1.287568,3.285200,1.003920,0.932240,2.495398,1.019696,0.929520
5,1.092500,1.289701,3.233040,0.995440,0.932400,2.506436,1.015866,0.930400


***** Running Evaluation *****
  Num examples = 12500
  Batch size = 32
Saving model checkpoint to /content/distilbert-base-uncased-cls/checkpoint-782
Configuration saved in /content/distilbert-base-uncased-cls/checkpoint-782/config.json
Model weights saved in /content/distilbert-base-uncased-cls/checkpoint-782/pytorch_model.bin
tokenizer config file saved in /content/distilbert-base-uncased-cls/checkpoint-782/tokenizer_config.json
Special tokens file saved in /content/distilbert-base-uncased-cls/checkpoint-782/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 12500
  Batch size = 32
Saving model checkpoint to /content/distilbert-base-uncased-cls/checkpoint-1564
Configuration saved in /content/distilbert-base-uncased-cls/checkpoint-1564/config.json
Model weights saved in /content/distilbert-base-uncased-cls/checkpoint-1564/pytorch_model.bin
tokenizer config file saved in /content/distilbert-base-uncased-cls/checkpoint-1564/tokenizer_config.json
Special tokens file

TrainOutput(global_step=3910, training_loss=1.2528266457950368, metrics={'train_runtime': 6633.5877, 'train_samples_per_second': 18.843, 'train_steps_per_second': 0.589, 'total_flos': 1.6560196608e+16, 'train_loss': 1.2528266457950368, 'epoch': 5.0})

# Fine tune (regression)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1).to(device)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

In [ ]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)
tokens_val = tokenizer.batch_encode_plus(
    val_text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)

In [ ]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["label"] = torch.tensor([self.labels[idx]]).to(torch.float32)
        return item

    def __len__(self):
        return len(self.labels)
    
train_dataset = Data(tokens_train, y_train)
val_dataset = Data(tokens_val, y_val)

In [ ]:
def compute_metrics(pred):
    labels = np.round(pred.label_ids.flatten())
    preds = pred.predictions.flatten()
    mse, mae, acc = reg_cls_metrics(labels, preds)
    return {'mse': mse, 'mae': mae, 'acc': acc}

In [ ]:
training_args = TrainingArguments(
    output_dir = '/content/distilbert-base-uncased-cls-reg', #Выходной каталог
    num_train_epochs = 5, #Кол-во эпох для обучения
    per_device_train_batch_size = 32, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 32, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = '/content/logs-reg', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=SEED)

In [ ]:
class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0][:, 0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
trainer = RegressionTrainer(model=model,
                            tokenizer = tokenizer,
                            args = training_args,
                            train_dataset = train_dataset,
                            eval_dataset = val_dataset,
                            compute_metrics = compute_metrics)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 25000
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3910
  Number of trainable parameters = 66954241


Epoch,Training Loss,Validation Loss,Mse,Mae,Acc
1,13.151400,12.166946,12.151390,3.281101,0.504640
2,12.062900,12.140621,12.127940,3.279782,0.504640
3,12.058100,12.140239,12.127345,3.279831,0.504640
4,12.059200,12.141824,12.126150,3.280005,0.504640
5,12.051600,12.140584,12.123675,3.279674,0.504640


***** Running Evaluation *****
  Num examples = 12500
  Batch size = 32
Saving model checkpoint to /content/distilbert-base-uncased-cls-reg/checkpoint-782
Configuration saved in /content/distilbert-base-uncased-cls-reg/checkpoint-782/config.json
Model weights saved in /content/distilbert-base-uncased-cls-reg/checkpoint-782/pytorch_model.bin
tokenizer config file saved in /content/distilbert-base-uncased-cls-reg/checkpoint-782/tokenizer_config.json
Special tokens file saved in /content/distilbert-base-uncased-cls-reg/checkpoint-782/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 12500
  Batch size = 32
Saving model checkpoint to /content/distilbert-base-uncased-cls-reg/checkpoint-1564
Configuration saved in /content/distilbert-base-uncased-cls-reg/checkpoint-1564/config.json
Model weights saved in /content/distilbert-base-uncased-cls-reg/checkpoint-1564/pytorch_model.bin
tokenizer config file saved in /content/distilbert-base-uncased-cls-reg/checkpoint-1564/token

TrainOutput(global_step=3910, training_loss=12.276623691256393, metrics={'train_runtime': 7227.5044, 'train_samples_per_second': 17.295, 'train_steps_per_second': 0.541, 'total_flos': 1.6558129536e+16, 'train_loss': 12.276623691256393, 'epoch': 5.0})

# Fine tuning final model

In [ ]:
EPOCHS = 5
LR = 1e-6
WD = 0.01

class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)
        
def compute_metrics(pred):
    labels = pred.label_ids.flatten()
    labels = np.array([id2labels[i] for i in labels])
    raw_preds = pred.predictions
    preds = raw_preds.argmax(-1)
    preds = np.array([id2labels[i] for i in preds])
    mse, mae, acc = reg_cls_metrics(labels, preds)

    preds_w = np.array([sum([id2labels[i] * w for i, w in enumerate(softmax(arr))]) for arr in raw_preds])
    mse_w, mae_w, acc_w = reg_cls_metrics(labels, preds_w)
    return {'mse': mse, 'mae': mae, 'acc': acc, 'mse_w': mse_w, 'mae_w': mae_w, 'acc_w': acc_w}
    
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=8).to(device)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

tokens_train = tokenizer.batch_encode_plus(
    train_text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)
tokens_val = tokenizer.batch_encode_plus(
    val_text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)

labels2id = {1:0, 2:1, 3:2, 4:3, 7:4, 8:5, 9:6, 10:7}
id2labels = {0:1, 1:2, 2:3, 3:4, 4:7, 5:8, 6:9, 7:10}

y_train_cls = np.array([labels2id[i] for i in y_train])
y_val_cls = np.array([labels2id[i] for i in y_val])
    
train_dataset = Data(tokens_train, y_train_cls)
val_dataset = Data(tokens_val, y_val_cls)

training_args = TrainingArguments(
    output_dir = '/content/distilbert-base-uncased-cls', #Выходной каталог
    num_train_epochs = EPOCHS, #Кол-во эпох для обучения
    per_device_train_batch_size = 32, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 32, #Размер пакета для каждого устройства во время валидации
    weight_decay = WD, #Понижение весов
    logging_dir = '/content/logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = LR, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=SEED)

trainer = Trainer(model=model,
                tokenizer = tokenizer,
                args = training_args,
                train_dataset = train_dataset,
                eval_dataset = val_dataset,
                compute_metrics = compute_metrics)



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Mse,Mae,Acc,Mse W,Mae W,Acc W
1,1.934900,1.719828,8.368160,1.809440,0.874160,6.322771,2.219115,0.856800
2,1.639000,1.559535,6.277280,1.515680,0.887920,4.306636,1.657795,0.886320
3,1.537700,1.496121,5.373840,1.377840,0.896800,3.817031,1.519077,0.890800
4,1.495600,1.469862,5.165920,1.341120,0.900400,3.621419,1.451766,0.896000


Epoch,Training Loss,Validation Loss,Mse,Mae,Acc,Mse W,Mae W,Acc W
1,1.934900,1.719828,8.368160,1.809440,0.874160,6.322771,2.219115,0.856800
2,1.639000,1.559535,6.277280,1.515680,0.887920,4.306636,1.657795,0.886320
3,1.537700,1.496121,5.373840,1.377840,0.896800,3.817031,1.519077,0.890800
4,1.495600,1.469862,5.165920,1.341120,0.900400,3.621419,1.451766,0.896000
5,1.478700,1.462712,5.141760,1.335360,0.901920,3.567168,1.430436,0.895920


TrainOutput(global_step=3910, training_loss=1.6171887800211797, metrics={'train_runtime': 7025.0071, 'train_samples_per_second': 17.794, 'train_steps_per_second': 0.557, 'total_flos': 1.6560196608e+16, 'train_loss': 1.6171887800211797, 'epoch': 5.0})

In [ ]:
EPOCHS = 5
LR = 1e-4
WD = 0.01

class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)
        
def compute_metrics(pred):
    labels = pred.label_ids.flatten()
    labels = np.array([id2labels[i] for i in labels])
    raw_preds = pred.predictions
    preds = raw_preds.argmax(-1)
    preds = np.array([id2labels[i] for i in preds])
    mse, mae, acc = reg_cls_metrics(labels, preds)

    preds_w = np.array([sum([id2labels[i] * w for i, w in enumerate(softmax(arr))]) for arr in raw_preds])
    mse_w, mae_w, acc_w = reg_cls_metrics(labels, preds_w)
    return {'mse': mse, 'mae': mae, 'acc': acc, 'mse_w': mse_w, 'mae_w': mae_w, 'acc_w': acc_w}
    
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=8).to(device)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

tokens_train = tokenizer.batch_encode_plus(
    train_text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)
tokens_val = tokenizer.batch_encode_plus(
    val_text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)

labels2id = {1:0, 2:1, 3:2, 4:3, 7:4, 8:5, 9:6, 10:7}
id2labels = {0:1, 1:2, 2:3, 3:4, 4:7, 5:8, 6:9, 7:10}

y_train_cls = np.array([labels2id[i] for i in y_train])
y_val_cls = np.array([labels2id[i] for i in y_val])
    
train_dataset = Data(tokens_train, y_train_cls)
val_dataset = Data(tokens_val, y_val_cls)

training_args = TrainingArguments(
    output_dir = '/content/distilbert-base-uncased-cls', #Выходной каталог
    num_train_epochs = EPOCHS, #Кол-во эпох для обучения
    per_device_train_batch_size = 32, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 32, #Размер пакета для каждого устройства во время валидации
    weight_decay = WD, #Понижение весов
    logging_dir = '/content/logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = LR, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=SEED)

trainer = Trainer(model=model,
                tokenizer = tokenizer,
                args = training_args,
                train_dataset = train_dataset,
                eval_dataset = val_dataset,
                compute_metrics = compute_metrics)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Mse,Mae,Acc,Mse W,Mae W,Acc W
1,1.476100,1.339078,4.339520,1.164640,0.908800,3.065161,1.189497,0.920320
2,1.148500,1.287695,3.430560,1.032160,0.921280,2.738172,1.057487,0.922800
3,0.889900,1.372527,3.131680,0.995520,0.926320,2.680583,1.029467,0.923040


KeyboardInterrupt: ignored

In [ ]:
EPOCHS = 5
LR = 1e-5
WD = 0.001

class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)
        
def compute_metrics(pred):
    labels = pred.label_ids.flatten()
    labels = np.array([id2labels[i] for i in labels])
    raw_preds = pred.predictions
    preds = raw_preds.argmax(-1)
    preds = np.array([id2labels[i] for i in preds])
    mse, mae, acc = reg_cls_metrics(labels, preds)

    preds_w = np.array([sum([id2labels[i] * w for i, w in enumerate(softmax(arr))]) for arr in raw_preds])
    mse_w, mae_w, acc_w = reg_cls_metrics(labels, preds_w)
    return {'mse': mse, 'mae': mae, 'acc': acc, 'mse_w': mse_w, 'mae_w': mae_w, 'acc_w': acc_w}
    
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=8).to(device)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

tokens_train = tokenizer.batch_encode_plus(
    train_text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)
tokens_val = tokenizer.batch_encode_plus(
    val_text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)

labels2id = {1:0, 2:1, 3:2, 4:3, 7:4, 8:5, 9:6, 10:7}
id2labels = {0:1, 1:2, 2:3, 3:4, 4:7, 5:8, 6:9, 7:10}

y_train_cls = np.array([labels2id[i] for i in y_train])
y_val_cls = np.array([labels2id[i] for i in y_val])
    
train_dataset = Data(tokens_train, y_train_cls)
val_dataset = Data(tokens_val, y_val_cls)

training_args = TrainingArguments(
    output_dir = '/content/distilbert-base-uncased-cls', #Выходной каталог
    num_train_epochs = EPOCHS, #Кол-во эпох для обучения
    per_device_train_batch_size = 32, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 32, #Размер пакета для каждого устройства во время валидации
    weight_decay = WD, #Понижение весов
    logging_dir = '/content/logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = LR, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=SEED)

trainer = Trainer(model=model,
                tokenizer = tokenizer,
                args = training_args,
                train_dataset = train_dataset,
                eval_dataset = val_dataset,
                compute_metrics = compute_metrics)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Mse,Mae,Acc,Mse W,Mae W,Acc W
1,1.521000,1.360652,4.116800,1.156480,0.920880,2.886591,1.168523,0.919040
2,1.302100,1.315197,3.336240,1.044240,0.927760,2.615301,1.097297,0.925440


Epoch,Training Loss,Validation Loss,Mse,Mae,Acc,Mse W,Mae W,Acc W
1,1.521000,1.360652,4.116800,1.156480,0.920880,2.886591,1.168523,0.919040
2,1.302100,1.315197,3.336240,1.044240,0.927760,2.615301,1.097297,0.925440
3,1.208300,1.288982,3.324720,1.016560,0.931120,2.524922,1.043496,0.927840


In [ ]:
EPOCHS = 15
LR = 1e-5
WD = 0.001

class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)
        
def compute_metrics(pred):
    labels = pred.label_ids.flatten()
    labels = np.array([id2labels[i] for i in labels])
    raw_preds = pred.predictions
    preds = raw_preds.argmax(-1)
    preds = np.array([id2labels[i] for i in preds])
    mse, mae, acc = reg_cls_metrics(labels, preds)

    preds_w = np.array([sum([id2labels[i] * w for i, w in enumerate(softmax(arr))]) for arr in raw_preds])
    mse_w, mae_w, acc_w = reg_cls_metrics(labels, preds_w)

    preds_avg = np.array([(i+j)/2 for i, j in zip(preds, preds_w)])
    mse_avg, mae_avg, acc_avg = reg_cls_metrics(labels, preds_avg)

    mse_w_int, mae_w_int, _ = reg_cls_metrics(labels, preds_w.round())
    mse_avg_int, mae_avg_int, _ = reg_cls_metrics(labels, preds_avg.round())

    return {'mse': mse, 'mae': mae, 'acc': acc, 
            'mse_w': mse_w, 'mae_w': mae_w, 'acc_w': acc_w,
            'mse_avg': mse_avg, 'mae_avg': mae_avg, 'acc_avg': acc_avg, 
            'mse_w_int': mse_w_int, 'mae_w_int': mae_w_int,
            'mse_avg_int': mse_avg_int, 'mae_avg_int': mae_avg_int}
    
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=8).to(device)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

tokens_train = tokenizer.batch_encode_plus(
    train_text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)
tokens_val = tokenizer.batch_encode_plus(
    val_text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)

labels2id = {1:0, 2:1, 3:2, 4:3, 7:4, 8:5, 9:6, 10:7}
id2labels = {0:1, 1:2, 2:3, 3:4, 4:7, 5:8, 6:9, 7:10}

y_train_cls = np.array([labels2id[i] for i in y_train])
y_val_cls = np.array([labels2id[i] for i in y_val])
    
train_dataset = Data(tokens_train, y_train_cls)
val_dataset = Data(tokens_val, y_val_cls)

training_args = TrainingArguments(
    output_dir = '/content/distilbert-base-uncased-cls', #Выходной каталог
    num_train_epochs = EPOCHS, #Кол-во эпох для обучения
    per_device_train_batch_size = 32, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 32, #Размер пакета для каждого устройства во время валидации
    weight_decay = WD, #Понижение весов
    logging_dir = '/content/logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = LR, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=SEED)

trainer = Trainer(model=model,
                tokenizer = tokenizer,
                args = training_args,
                train_dataset = train_dataset,
                eval_dataset = val_dataset,
                compute_metrics = compute_metrics)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Mse,Mae,Acc,Mse W,Mae W,Acc W,Mse Avg,Mae Avg,Acc Avg,Mse W Int,Mae W Int,Mse Avg Int,Mae Avg Int
1,1.519400,1.360474,4.164080,1.161520,0.920080,2.896933,1.169314,0.918560,3.265112,1.138517,0.920160,3.002560,1.168960,3.346640,1.077040
2,1.295000,1.309393,3.256560,1.031920,0.929120,2.591414,1.083827,0.927520,2.772669,1.046022,0.929120,2.669280,1.045280,2.914400,1.012800
3,1.186200,1.297228,3.544880,1.035280,0.928160,2.617359,1.037203,0.924560,2.922566,1.025768,0.928000,2.692560,0.984080,3.053200,0.987280
4,1.091000,1.298633,3.088800,0.979840,0.932560,2.464736,1.010111,0.930560,2.668668,0.989934,0.932640,2.552320,0.958560,2.814160,0.965520
5,1.000300,1.321527,2.917120,0.964160,0.932880,2.433807,1.012178,0.932400,2.582883,0.984729,0.933040,2.504400,0.964080,2.712560,0.957520
6,0.925400,1.354620,3.165200,0.981840,0.930320,2.603935,0.996020,0.928880,2.804143,0.985872,0.930320,2.692480,0.949120,2.946960,0.968080
7,0.851000,1.401606,3.148560,0.982000,0.930000,2.628450,1.004280,0.927520,2.812040,0.990357,0.929680,2.724720,0.968080,2.940720,0.972240
8,0.791300,1.459038,3.088720,0.978640,0.929120,2.645438,1.000118,0.927440,2.801825,0.987420,0.928880,2.736960,0.968800,2.922800,0.972240
9,0.734500,1.502671,3.177680,0.980880,0.931040,2.678766,0.988723,0.930160,2.863589,0.982299,0.930960,2.774080,0.957440,2.989200,0.970960
10,0.687600,1.550816,3.204880,0.991440,0.929440,2.714683,0.999468,0.928480,2.898093,0.993180,0.929680,2.818480,0.971600,3.016000,0.980640


TrainOutput(global_step=11730, training_loss=0.8652653947002544, metrics={'train_runtime': 20583.6655, 'train_samples_per_second': 18.218, 'train_steps_per_second': 0.57, 'total_flos': 4.9680589824e+16, 'train_loss': 0.8652653947002544, 'epoch': 15.0})

In [ ]:
EPOCHS = 15
LR = 5e-6
WD = 0.001

class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)
        
def compute_metrics(pred):
    labels = pred.label_ids.flatten()
    labels = np.array([id2labels[i] for i in labels])
    raw_preds = pred.predictions
    preds = raw_preds.argmax(-1)
    preds = np.array([id2labels[i] for i in preds])
    mse, mae, acc = reg_cls_metrics(labels, preds)

    preds_w = np.array([sum([id2labels[i] * w for i, w in enumerate(softmax(arr))]) for arr in raw_preds])
    mse_w, mae_w, acc_w = reg_cls_metrics(labels, preds_w)

    preds_avg = np.array([(i+j)/2 for i, j in zip(preds, preds_w)])
    mse_avg, mae_avg, acc_avg = reg_cls_metrics(labels, preds_avg)

    mse_w_int, mae_w_int, _ = reg_cls_metrics(labels, preds_w.round())
    mse_avg_int, mae_avg_int, _ = reg_cls_metrics(labels, preds_avg.round())

    return {'mse': mse, 'mae': mae, 'acc': acc, 
            'mse_w': mse_w, 'mae_w': mae_w, 'acc_w': acc_w,
            'mse_avg': mse_avg, 'mae_avg': mae_avg, 'acc_avg': acc_avg, 
            'mse_w_int': mse_w_int, 'mae_w_int': mae_w_int,
            'mse_avg_int': mse_avg_int, 'mae_avg_int': mae_avg_int}
    
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=8).to(device)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

tokens_train = tokenizer.batch_encode_plus(
    train_text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)
tokens_val = tokenizer.batch_encode_plus(
    val_text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)

labels2id = {1:0, 2:1, 3:2, 4:3, 7:4, 8:5, 9:6, 10:7}
id2labels = {0:1, 1:2, 2:3, 3:4, 4:7, 5:8, 6:9, 7:10}

y_train_cls = np.array([labels2id[i] for i in y_train])
y_val_cls = np.array([labels2id[i] for i in y_val])
    
train_dataset = Data(tokens_train, y_train_cls)
val_dataset = Data(tokens_val, y_val_cls)

training_args = TrainingArguments(
    output_dir = '/content/distilbert-base-uncased-cls', #Выходной каталог
    num_train_epochs = EPOCHS, #Кол-во эпох для обучения
    per_device_train_batch_size = 32, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 32, #Размер пакета для каждого устройства во время валидации
    weight_decay = WD, #Понижение весов
    logging_dir = '/content/logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = LR, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=SEED)

trainer = Trainer(model=model,
                tokenizer = tokenizer,
                args = training_args,
                train_dataset = train_dataset,
                eval_dataset = val_dataset,
                compute_metrics = compute_metrics)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Mse,Mae,Acc,Mse W,Mae W,Acc W,Mse Avg,Mae Avg,Acc Avg,Mse W Int,Mae W Int,Mse Avg Int,Mae Avg Int
1,1.604400,1.402119,4.667600,1.247920,0.913360,3.096020,1.238478,0.912320,3.514774,1.198278,0.913440,3.190320,1.235920,3.510960,1.125040
2,1.362800,1.345776,3.752160,1.112800,0.921520,2.836456,1.162165,0.919040,3.080064,1.116642,0.921520,2.932000,1.147680,3.175920,1.071600
3,1.294000,1.314241,3.610960,1.078480,0.926800,2.629744,1.097493,0.925120,2.916464,1.071661,0.926720,2.718160,1.069680,3.028240,1.023760
4,1.235900,1.306050,3.361760,1.038720,0.929680,2.560222,1.067474,0.927200,2.795299,1.041788,0.929600,2.652800,1.024320,2.927440,1.008080
5,1.182200,1.293359,3.204160,1.007200,0.931600,2.498574,1.049725,0.929600,2.707319,1.019914,0.931680,2.583040,1.002720,2.846160,0.985680
6,1.139100,1.303916,3.235520,1.005280,0.929600,2.554853,1.041597,0.927200,2.774148,1.016880,0.929680,2.645760,0.994560,2.909280,0.983040
7,1.092600,1.312677,3.511840,1.032800,0.928800,2.670958,1.034247,0.924640,2.962156,1.026708,0.928320,2.769280,0.988160,3.099360,0.992960
8,1.057700,1.315493,3.220560,0.998960,0.929600,2.574995,1.026688,0.928000,2.790137,1.008385,0.929520,2.654720,0.977120,2.936800,0.980640
9,1.025000,1.317208,3.266400,0.995840,0.930160,2.595534,1.016100,0.928240,2.827602,1.001231,0.930240,2.692480,0.970560,2.960960,0.973440


KeyboardInterrupt: ignored

# Final evaluate

In [ ]:
train_text_full = np.concatenate([train_text, val_text])
y_train_full = np.concatenate([y_train, y_val])

In [ ]:
EPOCHS = 5
LR = 1e-5
WD = 0.001

class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)
        
def compute_metrics(pred):
    labels = pred.label_ids.flatten()
    labels = np.array([id2labels[i] for i in labels])
    raw_preds = pred.predictions
    preds = raw_preds.argmax(-1)
    preds = np.array([id2labels[i] for i in preds])
    mse, mae, acc = reg_cls_metrics(labels, preds)

    preds_w = np.array([sum([id2labels[i] * w for i, w in enumerate(softmax(arr))]) for arr in raw_preds])
    mse_w, mae_w, acc_w = reg_cls_metrics(labels, preds_w)

    preds_avg = np.array([(i+j)/2 for i, j in zip(preds, preds_w)])
    mse_avg, mae_avg, acc_avg = reg_cls_metrics(labels, preds_avg)

    mse_w_int, mae_w_int, _ = reg_cls_metrics(labels, preds_w.round())
    mse_avg_int, mae_avg_int, _ = reg_cls_metrics(labels, preds_avg.round())

    return {'mse': mse, 'mae': mae, 'acc': acc, 
            'mse_w': mse_w, 'mae_w': mae_w, 'acc_w': acc_w,
            'mse_avg': mse_avg, 'mae_avg': mae_avg, 'acc_avg': acc_avg, 
            'mse_w_int': mse_w_int, 'mae_w_int': mae_w_int,
            'mse_avg_int': mse_avg_int, 'mae_avg_int': mae_avg_int}
    
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=8).to(device)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

tokens_train_full = tokenizer.batch_encode_plus(
    train_text_full.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)
tokens_test = tokenizer.batch_encode_plus(
    test_text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)

labels2id = {1:0, 2:1, 3:2, 4:3, 7:4, 8:5, 9:6, 10:7}
id2labels = {0:1, 1:2, 2:3, 3:4, 4:7, 5:8, 6:9, 7:10}

y_train_full_cls = np.array([labels2id[i] for i in y_train_full])
y_test_cls = np.array([labels2id[i] for i in y_test])
    
train_full_dataset = Data(tokens_train_full, y_train_full_cls)
test_dataset = Data(tokens_test, y_test_cls)

training_args = TrainingArguments(
    output_dir = '/content/distilbert-base-uncased-cls', #Выходной каталог
    num_train_epochs = EPOCHS, #Кол-во эпох для обучения
    per_device_train_batch_size = 32, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 32, #Размер пакета для каждого устройства во время валидации
    weight_decay = WD, #Понижение весов
    logging_dir = '/content/logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = LR, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=SEED)

trainer = Trainer(model=model,
                tokenizer = tokenizer,
                args = training_args,
                train_dataset = train_full_dataset,
                eval_dataset = test_dataset,
                compute_metrics = compute_metrics)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Mse,Mae,Acc,Mse W,Mae W,Acc W,Mse Avg,Mae Avg,Acc Avg,Mse W Int,Mae W Int,Mse Avg Int,Mae Avg Int
1,1.463800,1.321453,4.072080,1.125360,0.925040,2.714463,1.108029,0.921920,3.142210,1.094149,0.925040,2.804480,1.076000,3.235520,1.036640


Epoch,Training Loss,Validation Loss,Mse,Mae,Acc,Mse W,Mae W,Acc W,Mse Avg,Mae Avg,Acc Avg,Mse W Int,Mae W Int,Mse Avg Int,Mae Avg Int
1,1.463800,1.321453,4.072080,1.125360,0.925040,2.714463,1.108029,0.921920,3.142210,1.094149,0.925040,2.804480,1.076000,3.235520,1.036640
2,1.261500,1.244213,3.317520,0.993840,0.936000,2.347465,1.007259,0.936320,2.640110,0.984126,0.936000,2.431200,0.955360,2.751280,0.936400
3,1.163500,1.226197,3.136640,0.953440,0.937520,2.324952,0.975992,0.935840,2.577352,0.954485,0.937600,2.395040,0.921120,2.715600,0.920720
4,1.094800,1.226417,3.104400,0.946320,0.935360,2.350280,0.974568,0.933120,2.596923,0.952995,0.935520,2.435200,0.921920,2.743120,0.921360
5,1.051000,1.232837,3.125520,0.946160,0.937360,2.348415,0.954687,0.935920,2.616845,0.944351,0.937440,2.432080,0.901680,2.768080,0.919280


TrainOutput(global_step=5860, training_loss=1.2069243447365614, metrics={'train_runtime': 9629.1539, 'train_samples_per_second': 19.472, 'train_steps_per_second': 0.609, 'total_flos': 2.4840294912e+16, 'train_loss': 1.2069243447365614, 'epoch': 5.0})

In [ ]:
preds = trainer.predict(test_dataset).predictions

In [ ]:
softmax(preds, axis=-1)

array([[1.39110703e-02, 4.62415479e-02, 1.79338396e-01, ...,
        8.54830164e-03, 2.95900623e-03, 4.13391739e-03],
       [1.40812490e-02, 2.53013708e-02, 1.40481696e-01, ...,
        5.32333963e-02, 1.12186419e-02, 1.24833416e-02],
       [1.18795652e-02, 2.35717855e-02, 9.42182392e-02, ...,
        9.32840630e-02, 2.08880901e-02, 1.39411315e-02],
       ...,
       [9.00902152e-01, 6.55059516e-02, 2.02105157e-02, ...,
        1.01747585e-03, 7.29460327e-04, 2.22341786e-03],
       [2.53424561e-03, 2.23011058e-03, 1.62485242e-03, ...,
        1.37565553e-01, 2.11420968e-01, 6.06024325e-01],
       [2.15396145e-03, 1.75934145e-03, 1.28248346e-03, ...,
        8.98417011e-02, 1.55613959e-01, 7.29256690e-01]], dtype=float32)

In [ ]:
parameters = {'alpha': np.logspace(-8, 2, num=100)}
model = Ridge(random_state=42)
result = GridSearchCV(model, parameters, n_jobs=-1, cv=5, 
                      scoring='neg_mean_absolute_error').fit(preds, y_test)
print(f'MAE: {-result.best_score_}')
for _, alpha_best in result.best_params_.items():
    pass
print(f'alpha_best: {alpha_best}')

MAE: 1.0740163829612732
alpha_best: 1e-08


In [ ]:
parameters = {'alpha': np.logspace(-8, 2, num=100)}
model = Ridge(random_state=42)
result = GridSearchCV(model, parameters, n_jobs=-1, cv=5, 
                      scoring='neg_mean_absolute_error').fit(softmax(preds, axis=-1), y_test)
print(f'MAE: {-result.best_score_}')
for _, alpha_best in result.best_params_.items():
    pass
print(f'alpha_best: {alpha_best}')

MAE: 1.006515598602295
alpha_best: 1e-08


In [ ]:
y_test_bin = np.where(y_test > 5, 1, 0)

parameters = {'alpha': np.logspace(-8, 2, num=100)}
model = RidgeClassifier(random_state=42)
result = GridSearchCV(model, parameters, n_jobs=-1, cv=5, 
                      scoring='accuracy').fit(preds, y_test_bin)
print(f'accuracy: {result.best_score_}')
for _, alpha_best in result.best_params_.items():
    pass
print(f'alpha_best: {alpha_best}')

accuracy: 0.9379200000000001
alpha_best: 62.80291441834247


In [ ]:
y_test_bin = np.where(y_test > 5, 1, 0)

parameters = {'alpha': np.logspace(-8, 2, num=100)}
model = RidgeClassifier(random_state=42)
result = GridSearchCV(model, parameters, n_jobs=-1, cv=5, 
                      scoring='accuracy').fit(softmax(preds, axis=-1), y_test_bin)
print(f'accuracy: {result.best_score_}')
for _, alpha_best in result.best_params_.items():
    pass
print(f'alpha_best: {alpha_best}')

accuracy: 0.938
alpha_best: 39.44206059437648


# Final fit

In [10]:
text = np.concatenate([train_text, val_text, test_text])
y = np.concatenate([y_train, y_val, y_test])

In [11]:
EPOCHS = 5
LR = 1e-5
WD = 0.001

class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)
        
def compute_metrics(pred):
    labels = pred.label_ids.flatten()
    labels = np.array([id2labels[i] for i in labels])
    raw_preds = pred.predictions
    preds = raw_preds.argmax(-1)
    preds = np.array([id2labels[i] for i in preds])
    mse, mae, acc = reg_cls_metrics(labels, preds)

    return {'mse': mse, 'mae': mae, 'acc': acc}
    
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=8).to(device)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

tokens = tokenizer.batch_encode_plus(
    text.astype('str'),
    max_length = 512,
    padding = 'max_length',
    truncation = True
)

labels2id = {1:0, 2:1, 3:2, 4:3, 7:4, 8:5, 9:6, 10:7}
id2labels = {0:1, 1:2, 2:3, 3:4, 4:7, 5:8, 6:9, 7:10}

y = np.array([labels2id[i] for i in y])
    
dataset = Data(tokens, y)

training_args = TrainingArguments(
    output_dir = '/content/distilbert-base-uncased-cls', #Выходной каталог
    num_train_epochs = EPOCHS, #Кол-во эпох для обучения
    per_device_train_batch_size = 32, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 32, #Размер пакета для каждого устройства во время валидации
    weight_decay = WD, #Понижение весов
    logging_dir = '/content/logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = LR, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=SEED)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

In [12]:
trainer = Trainer(model=model,
                tokenizer = tokenizer,
                args = training_args,
                train_dataset = dataset,
                eval_dataset = dataset,
                compute_metrics = compute_metrics)

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Mse,Mae,Acc
1,1.426400,1.229917,2.955400,0.945280,0.946580
2,1.225600,1.097695,2.042500,0.747980,0.964340
3,1.121900,1.018625,1.637560,0.663040,0.974060
4,1.058000,0.981679,1.525220,0.632540,0.978360
5,1.017200,0.951037,1.339720,0.584480,0.981220


TrainOutput(global_step=7815, training_loss=1.169823515849928, metrics={'train_runtime': 16324.8512, 'train_samples_per_second': 15.314, 'train_steps_per_second': 0.479, 'total_flos': 3.3120393216e+16, 'train_loss': 1.169823515849928, 'epoch': 5.0})

In [14]:
trainer.save_model('model_nlp')

In [15]:
!zip -r '/content/model_nlp.zip' '/content/model_nlp'

  adding: content/model_nlp/ (stored 0%)
  adding: content/model_nlp/tokenizer_config.json (deflated 45%)
  adding: content/model_nlp/config.json (deflated 54%)
  adding: content/model_nlp/training_args.bin (deflated 48%)
  adding: content/model_nlp/pytorch_model.bin (deflated 8%)
  adding: content/model_nlp/special_tokens_map.json (deflated 42%)
  adding: content/model_nlp/vocab.txt (deflated 53%)


In [16]:
from google.colab import files

files.download('/content/model_nlp.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>